In [27]:
import re
import requests
from urllib.parse import quote
import pandas as pd

# List of cities to scrape
cities = [
    "Makati",
    "Muntinlupa",
    "Pasig",
    "Bacoor",
    "Trece Martires",
    "Calamba",
    "Batangas City",
    "Cainta",
    "Rodriguez",
    "San Fernando",
    "Mandaluyong",
    "Pasig",
    "Taguig",
    "Manila"
]

base_url = "https://www.lamudi.com.ph/buy/{}/house/"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

# Regex pattern to extract prices
pattern = r'<div class="snippet__content__price">\s*([^<]+)\s*</div>'

# Create a list to store results
results = []

for city in cities:
    encoded_city = quote(city.lower().replace(" ", "-"))
    url = base_url.format(encoded_city)

    print(f"\n--- {city} ---")
    print(f"Fetching: {url}")

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        html = response.text

        prices = re.findall(pattern, html)
        prices = [p.strip() for p in prices][:30]  # limit to 10 samples

        if prices:
            for price in prices:
                results.append({
                    "City": city,
                    "Price": price
                })
        else:
            print("No prices found.")

    except requests.RequestException as e:
        print(f"Error fetching {city}: {e}")

# Convert results to a DataFrame
df = pd.DataFrame(results)

print("\n✅ Scraping complete! Here's a sample:")
print(df.head())


--- Makati ---
Fetching: https://www.lamudi.com.ph/buy/makati/house/

--- Muntinlupa ---
Fetching: https://www.lamudi.com.ph/buy/muntinlupa/house/

--- Pasig ---
Fetching: https://www.lamudi.com.ph/buy/pasig/house/

--- Bacoor ---
Fetching: https://www.lamudi.com.ph/buy/bacoor/house/

--- Trece Martires ---
Fetching: https://www.lamudi.com.ph/buy/trece-martires/house/

--- Calamba ---
Fetching: https://www.lamudi.com.ph/buy/calamba/house/

--- Batangas City ---
Fetching: https://www.lamudi.com.ph/buy/batangas-city/house/

--- Cainta ---
Fetching: https://www.lamudi.com.ph/buy/cainta/house/

--- Rodriguez ---
Fetching: https://www.lamudi.com.ph/buy/rodriguez/house/

--- San Fernando ---
Fetching: https://www.lamudi.com.ph/buy/san-fernando/house/

--- Mandaluyong ---
Fetching: https://www.lamudi.com.ph/buy/mandaluyong/house/

--- Pasig ---
Fetching: https://www.lamudi.com.ph/buy/pasig/house/

--- Taguig ---
Fetching: https://www.lamudi.com.ph/buy/taguig/house/

--- Manila ---
Fetching: 

In [28]:
# Convert prices to numeric
def clean_price(price_str):
    # Remove Peso sign, commas, spaces, and non-numeric characters
    clean = re.sub(r"[^\d.]", "", price_str)
    return float(clean) if clean else None

df["Price_clean"] = df["Price"].apply(clean_price)

print(df[["City", "Price", "Price_clean"]].head())

     City            Price   Price_clean
0  Makati  ₱ 1,500,000,000  1.500000e+09
1  Makati  ₱ 1,500,000,000  1.500000e+09
2  Makati  ₱ 1,700,000,000  1.700000e+09
3  Makati  ₱ 1,900,000,000  1.900000e+09
4  Makati    ₱ 175,000,000  1.750000e+08


In [29]:
df

,City,Price,Price_clean
0,Makati,"₱ 1,500,000,000",1.500000e+09
1,Makati,"₱ 1,500,000,000",1.500000e+09
2,Makati,"₱ 1,700,000,000",1.700000e+09
3,Makati,"₱ 1,900,000,000",1.900000e+09
4,Makati,"₱ 175,000,000",1.750000e+08
...,...,...,...
406,Manila,"₱ 7,800,000",7.800000e+06
407,Manila,"₱ 29,500,000",2.950000e+07
408,Manila,"₱ 29,000,000",2.900000e+07
409,Manila,"₱ 15,500,000",1.550000e+07


In [30]:
import plotly.graph_objects as go

# Drop missing values
df_plot = df.dropna(subset=["Price_clean"])

# Sort cities alphabetically (optional)
df_plot = df_plot.sort_values(by="City")

# Create a trace (one box per city)
fig = go.Figure()

for city in df_plot["City"].unique():
    fig.add_trace(go.Box(
        y=df_plot.loc[df_plot["City"] == city, "Price_clean"],
        name=city,
        # boxmean='sd',  # show mean and standard deviation
        # marker=dict(outliercolor='rgba(255, 0, 0, 0.6)', size=8)
    ))

# Customize layout
fig.update_layout(
    title="🏘️ Distribution of Property Prices by City",
    yaxis_title="Price (PHP)",
    xaxis_title="City",
    boxmode="group",
    template="plotly_white",
    height=600,
    width=1000,
    # boxmode="group",
    boxgap=0.0,         # 0 = no gap between categories (widest)
    boxgroupgap=0.01 
)
fig.update_layout(
       # small gap within groups
)
fig.show()


In [38]:
import numpy as np
import plotly.graph_objects as go
from scipy.stats import gaussian_kde

df_plot = df.dropna(subset=["Price_clean"]).sort_values(by="City")
cities = df_plot["City"].unique()

x_min, x_max = df_plot["Price_clean"].min(), df_plot["Price_clean"].max()
xs = np.linspace(x_min, x_max*0.05, 512)
xs = np.linspace(x_min, x_max, 512)

fig = go.Figure()
for c in cities:
    vals = df_plot.loc[df_plot["City"] == c, "Price_clean"].values
    kde = gaussian_kde(vals, bw_method=0.3)  # adjust 0.1–0.5 to tune smoothness
    fig.add_trace(go.Scatter(
        x=xs, y=kde(xs), mode="lines", name=c, opacity=0.9
        # fill="tozeroy"  # uncomment to fill under curves
    ))

fig.update_layout(
    title="🏘️ Property Price Density by City (KDE)",
    xaxis_title="Price (PHP)",
    yaxis_title="Density",
    template="plotly_white",
    height=600, width=1000,
    legend_title_text="City"
)
fig.show()


Old Code

In [5]:
import re
import requests

# URL for demo (you can replace it with your Lamudi link)
url = "https://www.lamudi.com.ph/buy/metro-manila/muntinlupa/alabang/house/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}

# Fetch page
response = requests.get(url, headers=headers)
html = response.text

# Regex pattern to match:
# <div class="snippet__content__price"> ₱ 45,000,000 </div>
pattern = r'<div class="snippet__content__price">\s*([^<]+)\s*</div>'

# Find all matches
prices = re.findall(pattern, html)

print("Extracted prices:")
for price in prices:
    print(price.strip())


Extracted prices:
₱ 45,000,000
₱ 63,000,000
₱ 105,000,000
₱ 95,000,000
₱ 185,000,000
₱ 90,000,000
₱ 63,000,000
₱ 63,000,000
₱ 95,000,000
₱ 125,000,000
₱ 61,813,675
₱ 98,000,989
₱ 60,000,000
₱ 65,000,000
₱ 120,000,000
₱ 105,000,000
₱ 65,000,000
₱ 420,000,000
₱ 87,990,000
₱ 170,000,000
₱ 200,000,000
₱ 149,900,000
₱ 49,839,570
₱ 20,500,000
₱ 58,000,000
₱ 135,000,000
₱ 88,000,000
₱ 92,500,000
₱ 80,000,000
₱ 410,000,000


In [8]:
import re
import requests
import pandas as pd

# --- Target URL ---
url = "https://www.lamudi.com.ph/buy/metro-manila/muntinlupa/taguig/house/"

# --- Headers to mimic a browser ---
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}

# --- Fetch page ---
response = requests.get(url, headers=headers)
html = response.text

# --- Regex patterns ---
price_pattern = r'<div class="snippet__content__price">\s*([^<]+)\s*</div>'
location_pattern = r'<span[^>]*data-test="snippet-content-location"[^>]*>\s*([^<]+)\s*</span>'

# --- Find all matches ---
prices = [p.strip() for p in re.findall(price_pattern, html)]
locations = [l.strip() for l in re.findall(location_pattern, html)]

# --- Align lists (in case of mismatch) ---
min_len = min(len(prices), len(locations))
prices, locations = prices[:min_len], locations[:min_len]

# --- Create DataFrame ---
df = pd.DataFrame({
    "Price": prices,
    "Location": locations
})

# --- Clean price column (optional numeric cleaning) ---
df["Clean_Price"] = (
    df["Price"]
    .str.replace("₱", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.strip()
)

# --- Display results ---
print(df.head())

# --- Optionally save to CSV ---
# df.to_csv("lamudi_prices.csv", index=False)
# print("\nData saved to lamudi_prices.csv")


           Price                Location Clean_Price
0    ₱ 1,384,600  Fort Bonifacio, Taguig     1384600
1   ₱ 85,000,000         Bicutan, Taguig    85000000
2   ₱ 48,000,000    Taguig, Metro Manila    48000000
3  ₱ 200,000,000   McKinley Hill, Taguig   200000000
4            ₱ 1   McKinley Hill, Taguig           1


In [9]:
df

,Price,Location,Clean_Price
0,"₱ 1,384,600","Fort Bonifacio, Taguig",1384600
1,"₱ 85,000,000","Bicutan, Taguig",85000000
2,"₱ 48,000,000","Taguig, Metro Manila",48000000
3,"₱ 200,000,000","McKinley Hill, Taguig",200000000
4,₱ 1,"McKinley Hill, Taguig",1
5,₱ 1,"McKinley Hill, Taguig",1
6,₱ 1,"Fort Bonifacio, Taguig",1
7,"₱ 50,000,000","Bicutan, Taguig",50000000
8,"₱ 198,000,000","Fort Bonifacio, Taguig",198000000
9,"₱ 196,000,000","Fort Bonifacio, Taguig",196000000


Scraping Numbeo

In [ ]:
import json

# Sample data (your snippet shortened for demo)
sample_data = {
  "city_id": 6512,
  "prices": [
    {
      "data_points": 70,
      "item_id": 14,
      "lowest_price": 6,
      "average_price": 8,
      "highest_price": 10,
      "item_name": "Bottle of Wine (Mid-Range), Markets"
    },
    {
      "data_points": 52,
      "item_id": 15,
      "lowest_price": 1,
      "average_price": 1.9053921568627454,
      "highest_price": 4,
      "item_name": "Domestic Beer (0.5 liter bottle), Markets"
    }
  ]
}

# Simulate what fetch_numbeo_prices() would return
data = sample_data

# Pretty-print output
for item in data.get("prices", []):
    print(
        f"{item['item_name']}: avg {item['average_price']} "
        f"(low {item['lowest_price']} – high {item['highest_price']})"
    )
